In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Q-dFjPmwh9EGXPEWeAUxygomRUp_mdrOtqrviuWCGBYk',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'networkintrusiondetection-donotdelete-pr-nwka2dravae0ni'
object_key = 'Train_data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# --- CHANGES ARE HERE ---
# 1. We load the data into a DataFrame with a clear name: 'train_df'
train_df = pd.read_csv(body)

# 2. We display the first 5 rows of our new DataFrame to inspect it.
train_df.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [2]:
# Get a concise summary of the dataframe.
# This will show us the column names, the number of non-null values, and the data type of each column.
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [3]:
# We need a special tool from the scikit-learn library to do this conversion.
from sklearn.preprocessing import LabelEncoder

# Create a new copy of our dataframe to work on, leaving the original untouched.
processed_df = train_df.copy()

# Identify the columns that contain text (we found these in the last step).
categorical_cols = ['protocol_type', 'service', 'flag']

# Loop through each of the text columns.
for col in categorical_cols:
    # Create a new LabelEncoder for this column.
    le = LabelEncoder()
    # The 'fit_transform' method learns all the unique text values and converts them to numbers.
    processed_df[col] = le.fit_transform(processed_df[col])

# Let's display the first 5 rows again to see the changes.
# Look at the 'protocol_type', 'service', and 'flag' columns. They should now contain numbers.
processed_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,19,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,41,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,46,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,1,22,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,22,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
# The 'class' column is our target variable. It tells us if an entry is an attack or normal.
# We will separate it from the rest of the data.

# 'X' will contain all columns EXCEPT for the 'class' column.
X = processed_df.drop('class', axis=1)

# 'y' will contain ONLY the 'class' column.
y = processed_df['class']

# Let's print the shape of our new variables to confirm.
print("Shape of our features (X):", X.shape)
print("Shape of our labels (y):", y.shape)

Shape of our features (X): (25192, 41)
Shape of our labels (y): (25192,)


In [5]:
# We need a special tool from scikit-learn to split our data.
from sklearn.model_selection import train_test_split

# This function will split our data into training and testing sets.
# test_size=0.2 means we will keep 20% of the data for testing.
# random_state=42 ensures that we get the same split every time we run the code, which is good for consistency.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Let's print the shapes of our new datasets to confirm the split was successful.
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (20153, 41)
Shape of X_test: (5039, 41)
Shape of y_train: (20153,)
Shape of y_test: (5039,)


In [6]:
# We will use the Decision Tree algorithm from scikit-learn.
from sklearn.tree import DecisionTreeClassifier

# 1. Create the model: We are creating a new, untrained Decision Tree model.
#    random_state=42 is used for reproducibility.
model = DecisionTreeClassifier(random_state=42)

# 2. Train the model: This is the most important step.
#    The 'fit' method teaches the model by showing it the training features (X_train)
#    and the corresponding correct answers (y_train).
model.fit(X_train, y_train)

# 3. Print a success message to confirm the training is complete.
print("Decision Tree model has been successfully trained!")

Decision Tree model has been successfully trained!


In [7]:
# We need the accuracy_score function to grade our model.
from sklearn.metrics import accuracy_score

# 1. Make predictions: We ask our trained model to predict the outcome for our test data (X_test).
y_pred = model.predict(X_test)

# 2. Calculate accuracy: We compare the model's predictions (y_pred) to the true answers (y_test).
#    The function returns a score, which we multiply by 100 to get a percentage.
accuracy = accuracy_score(y_test, y_pred) * 100

# 3. Print the final result.
print(f"The model's accuracy on the test data is: {accuracy:.2f}%")

The model's accuracy on the test data is: 99.58%
